In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# Load Yolo
net = cv2.dnn.readNet("../Yolov3_MulPatches/yolov3_training_final.weights", "../Yolov3_MulPatches/yolov3_testing.cfg")


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-k8sx3e60\opencv\modules\dnn\src\darknet\darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: ../Yolov3_MulPatches/yolov3_testing.cfg in function 'cv::dnn::dnn4_v20200609::readNetFromDarknet'


In [ ]:
classes = []
with open("../Yolov3_MulPatches/classes.txt", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
print(classes)

In [ ]:
# Loading image
img = cv2.imread("../Yolov3_MulPatches/104 ST low ac T1 S6.jpg")
#104 ST low ac T1 S6
img = cv2.resize(img, None, fx=0.1, fy=0.1)
height, width, channels = img.shape
#104 ST low ac T1 S6
plt.figure(figsize=(6,6))
plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
for b in blob:
    for n, img_blob in enumerate(b):
        cv2.imshow(str(n), img_blob)

net.setInput(blob)
outs = net.forward(output_layers)

In [ ]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
             # Rectangle coordinates

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
print(len(boxes))

In [ ]:

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)


In [ ]:
number_objects_detected = len(boxes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    x, y, w, h = boxes[i]
    label = classes[class_ids[i]]
    color = colors[class_ids[i]]
    print(label)
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, label, (x, y + 30), font, 1.5, (0, 0, 0), 2)


In [ ]:
cv2.imshow("Image", img)
#cv2.waitKey(0)
#cv2.imwrite('D:/ImagewithLabel1.jpg', img)
# plt.imshow(img)
# plt.show()
cv2.destroyAllWindows()
plt.figure(figsize=(7,7))
plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()